## Set Hypoparameters, Models, Loss, Optm, Scheduler, ...

In [3]:
#hypoparameters, settings
import torch
from utils.models import EffNet_basic
from utils.loss import focal_loss, CB_loss
from torch import optim
from torch.optim import lr_scheduler

batch_size = 32
random_seed = 77
lr = 1e-3
momentum = 0.9
weight_decay = 3e-5

device = torch.device('cuda')

model = EffNet_basic(backbone = "efficientnet-b6")
model_name = "eff_b6"
criterion = CB_loss
optimizer = optim.SGD(model.parameters(),lr=lr,momentum=momentum,weight_decay=weight_decay)
scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,5)

Loaded pretrained weights for efficientnet-b6


## Load K-folded dataset

In [4]:
import pickle
import itertools
from torchvision import transforms
from utils.dataset import MaskTrainDataset, MaskTestDataset, TestDataset, default_transform, MaskTrainDataset_Basic, MaskTestDataset_Basic 
from utils.utils import get_label
from torch.utils.data import DataLoader
from tqdm import tqdm

with open("fold_by_10_modified", "rb") as f:
    fold_by_10 = pickle.load(f)

with open("folds_eval", "rb") as f:
    folds_eval = pickle.load(f)

with open("fold_by_10_basic","rb") as f:
    folds_by_10_basic = pickle.load(f)

with open("samples_per_folds", "rb") as f:
    samples_per_folds = pickle.load(f)

with open("samples_per_folds_gender", "rb") as f:
    samples_per_folds_gender = pickle.load(f)

with open("samples_per_folds_age", "rb") as f:
    samples_per_folds_age = pickle.load(f)

with open("samples_per_folds_masked", "rb") as f:
    samples_per_folds_masked = pickle.load(f)
    
samples_per_folds_each = [samples_per_folds_masked, samples_per_folds_gender, samples_per_folds_age]


label_weights = []
for fold in samples_per_folds:
    label_weight = []
    num_fold = sum(fold)
    for label in fold:
        label_weight.append(label/num_fold)
    label_weights.append(label_weight)
    
folds = []
for i in range(10):
    train_list = list(itertools.chain.from_iterable(folds_eval[:i]))+list(itertools.chain.from_iterable(folds_eval[i+1:]))
    #train_list = folds_eval[:i]+folds_eval[i+1:]
    eval_list = folds_eval[i]
    
    train_dataset = MaskTrainDataset_Basic(train_list, default_transform())
    test_dataset = MaskTestDataset_Basic(eval_list, default_transform())
    
    train_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True,num_workers=4)
    test_loader = DataLoader(test_dataset,batch_size=batch_size,shuffle=False,num_workers=4)
    
    folds.append((train_loader,test_loader))

## Define Train

In [5]:
import time
import copy

def train(model,num_epochs,k):
    model.to(device)
    best_eval_acc = 0.0
    best_eval_loss = 1.0
    fold_start = time.time()
    for epoch in range(num_epochs):
        start = time.time()
        print(f'{epoch+1}/{num_epochs} epochs')
        print('-----------')
        train_loss, train_corrects, train_labels = 0,0,0
        eval_loss, eval_corrects, eval_labels = 0,0,0

        #train
        model.train()
        for inputs, labels in tqdm(folds[k][0]):    #iterate k th fold train dataloader 
            mask,gender,age = labels
            mask = mask.to(device)
            gender = gender.to(device)
            age = age//10
            age = age.to(device)
            labels = [mask,gender,age]
            optimizer.zero_grad()
            inputs = inputs.to(device)
            outputs = model(inputs)    #3,2,7
            loss = torch.tensor(0.0).to(device)
            
            for i,output in enumerate(outputs):
                if i == 2:
                    alpha = 5
                else:
                    alpha = 1
                _, preds = torch.max(output,1)
                num_of_classes = len(samples_per_folds_each[i][k])
                temp_loss= alpha*criterion(labels[i],output,samples_per_folds_each[i][k],num_of_classes)
                loss+=temp_loss
    
            loss.backward()
            optimizer.step()
            scheduler.step()
            train_loss += loss.item() * inputs.size(0)
            train_corrects += torch.sum(preds==labels[i].data)
            train_labels += len(labels[i])

        torch.save(model.state_dict(),f'{model_name}_{k}fold_imsi.pt')
        train_avg_loss = float(train_loss/train_labels)
        train_acc = float(train_corrects.double()/train_labels)

        print(f"train loss : {round(train_avg_loss,5)}")
        print(f"train acc : {round(train_acc,5)}")

        
        eval
        with torch.no_grad():
            model.eval()

            for inputs, labels in tqdm(folds[k][1]):    #iterate k th fold test dataloader
                mask,gender,age = labels
                mask = mask.to(device)
                gender = gender.to(device)
                age = age//10
                age = age.to(device)
                labels = [mask,gender,age]
                optimizer.zero_grad()
                inputs = inputs.to(device)
                outputs = model(inputs)    #3,2,7
                loss = torch.tensor(0.0).to(device)
                
                for i,output in enumerate(outputs):
                    
                    if i == 2:
                        alpha = 5
                    else:
                        alpha = 1
                        
                    _, preds = torch.max(output,1)
                    num_of_classes = len(samples_per_folds_each[i][k])
                    temp_loss= criterion(labels[i],output,samples_per_folds_each[i][k],num_of_classes)
                    ### SAMPLES PER FOLDS EACH 달라야함!!
                    loss+=alpha*temp_loss
                    eval_corrects += torch.sum(preds==labels[i].data)
                    eval_labels += len(labels[i])
                    
                eval_loss += loss.item() * inputs.size(0)
                
            eval_avg_loss = float(eval_loss/eval_labels)
            eval_acc = float(eval_corrects.double()/eval_labels)*100
            
            print(f"eval loss : {round(eval_avg_loss,5)}")
            print(f"eval acc : {round(eval_acc,5)}")

            if eval_avg_loss<=best_eval_loss:
                best_eval_loss = eval_avg_loss
                torch.save(model.state_dict(),f'{model_name}_{k}fold_best.pt')
                print(f"best model saved : '{model_name}_{k}fold_best.pt")

            time_spent = time.time() - start
            print(f"epoch train time : {round(time_spent//60)}min {round(time_spent%60)}sec")
    fold_time_spent = time.time() - fold_start
    print(f"fold train time : {round(time_spent//60)}min {round(time_spent%60)}sec")
    
def k_fold_train(num_epochs=15,k=10):
    for i in range(k):
        model = EffNet_basic(backbone = "efficientnet-b0")
        train(model=model,num_epochs=num_epochs,k=i)
    

In [6]:
#model.load_state_dict(torch.load("eff_b0_5fold_imsi.pt"))
train(model,50,5)

  0%|          | 0/532 [00:00<?, ?it/s]

1/50 epochs
-----------


  0%|          | 0/59 [00:00<?, ?it/s]

train loss : 1.83772
train acc : 0.5531


100%|██████████| 59/59 [00:15<00:00,  3.73it/s]


...

eval loss : 1.19155
eval acc : 88.71713
epoch train time : 8min 16sec
50/50 epochs
-----------


  0%|          | 0/59 [00:00<?, ?it/s]

train loss : 0.01627
train acc : 0.99906


100%|██████████| 59/59 [00:15<00:00,  3.77it/s]

eval loss : 1.17873
eval acc : 88.85928
epoch train time : 8min 15sec
fold train time : 8min 15sec


In [28]:
labels

NameError: name 'labels' is not defined

# Inference

In [11]:
import pandas as pd
import os
best_model = f'eff_b6_5fold_imsi.pt'
test_dir = '/opt/ml/stage1/eval'
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = default_transform()
dataset = TestDataset(image_paths, transform)
loader = DataLoader(dataset,shuffle=False)

print(f'loading {best_model} ...')

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = [[],[],[]]

submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
model.load_state_dict(torch.load(best_model))
model.to(device)
model.eval()

for images in tqdm(loader):
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        for i,elem in enumerate(pred):
            pred_each = elem.argmax(dim=-1)
            all_predictions[i].extend(pred_each)

all_predictions
# submission['ans'] = all_predictions
# submission.to_csv(f'{best_model}.csv', index=False)


# # 제출할 파일을 저장합니다.
# print('test inference is done!')

loading eff_b6_5fold_imsi.pt ...


100%|██████████| 12600/12600 [12:39<00:00, 16.59it/s]


[[tensor(2, device='cuda:0'),
  tensor(0, device='cuda:0'),
  tensor(2, device='cuda:0'),
  tensor(2, device='cuda:0'),
  tensor(2, device='cuda:0'),
  tensor(0, device='cuda:0'),
  tensor(1, device='cuda:0'),
  tensor(0, device='cuda:0'),
  tensor(0, device='cuda:0'),
  tensor(0, device='cuda:0'),
  tensor(2, device='cuda:0'),
  tensor(0, device='cuda:0'),
  tensor(1, device='cuda:0'),
  tensor(0, device='cuda:0'),
  tensor(0, device='cuda:0'),
  tensor(0, device='cuda:0'),
  tensor(0, device='cuda:0'),
  tensor(1, device='cuda:0'),
  tensor(1, device='cuda:0'),
  tensor(0, device='cuda:0'),
  tensor(2, device='cuda:0'),
  tensor(0, device='cuda:0'),
  tensor(0, device='cuda:0'),
  tensor(2, device='cuda:0'),
  tensor(0, device='cuda:0'),
  tensor(0, device='cuda:0'),
  tensor(2, device='cuda:0'),
  tensor(0, device='cuda:0'),
  tensor(0, device='cuda:0'),
  tensor(0, device='cuda:0'),
  tensor(0, device='cuda:0'),
  tensor(0, device='cuda:0'),
  tensor(0, device='cuda:0'),
  tensor(0

In [15]:
submission['masked'] = list(map(int,all_predictions[0]))
submission['gender'] = list(map(int,all_predictions[1]))
submission['age'] = list(map(int,all_predictions[2]))
submission['ans'] = submission['masked']*6+submission['gender']*3+submission['age']//3
submission['ans'].unique()

array([14,  1, 13, 12,  0,  7,  3,  4,  6, 16,  2,  9, 10, 15,  5,  8, 17,
       11])

In [16]:
submission.drop(['age','gender','masked'],axis=1).to_csv(f'b6_customized.csv', index=False)

## Ensemble

In [26]:
import collections
dfs = []
for i in range(6):
    df = pd.read_csv(f'output{i}.csv')
    dfs.append(df)
ans = []
for i,row in tqdm(df.iterrows()):
    d = collections.defaultdict(int)
    for j in range(6):
        d[dfs[j].loc[i]['ans']]+=1
    a = sorted(d.items(),reverse=True,key = lambda x:x[1])[0][0]
    ans.append(a)

12600it [00:12, 1005.99it/s]


In [27]:
df['ans']=ans
df.to_csv(f'ens.csv', index=False)

In [23]:
submission.head(20)

,ImageID,ans,masked,gender,age
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,14,2,0,6
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,1,0,0,5
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,13,2,0,5
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,13,2,0,4
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,12,2,0,2
5,7953c2e6e983bad91b89a0e4ad7eb69e6e43e56b.jpg,0,0,0,1
6,1903db7dcc1262d0441677afa422e6a8371e5b37.jpg,7,1,0,5
7,441419a874f4d031cd576850b68539ca7d35bedf.jpg,3,0,1,1
8,388856cd1ef99b1918273a827a75f2aff2478321.jpg,4,0,1,5
9,795ba8ccc769a3f9da6a897f75df6706b729345b.jpg,3,0,1,2


In [24]:
s_2 = pd.read_csv('/opt/ml/code/submission_eff_b6cb10.csv')
s_2.head(20)

,ImageID,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,13
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,1
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,13
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,13
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,12
5,7953c2e6e983bad91b89a0e4ad7eb69e6e43e56b.jpg,0
6,1903db7dcc1262d0441677afa422e6a8371e5b37.jpg,7
7,441419a874f4d031cd576850b68539ca7d35bedf.jpg,3
8,388856cd1ef99b1918273a827a75f2aff2478321.jpg,4
9,795ba8ccc769a3f9da6a897f75df6706b729345b.jpg,3


In [21]:
!pwd

/opt/ml/stage1/train
